In [1]:
import re, collections

def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab


In [3]:
filename = "extract.txt"
v = get_vocab(filename)

In [4]:
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

In [7]:
s = sorted(get_stats(v))
for line in s:
    print(line)

(',', '</w>')
('.', '</w>')
('0', '4')
('1', '6')
('4', '</w>')
('6', '0')
(';', '</w>')
('A', '</w>')
('A', 'S')
('A', 'T')
('A', 'l')
('A', 'n')
('C', 'e')
('C', 'o')
('C', 'á')
('D', 'E')
('D', 'i')
('E', '</w>')
('E', 'L')
('E', 'R')
('E', 'S')
('E', 'Y')
('E', 'l')
('E', 'n')
('E', 's')
('F', 'r')
('G', 'a')
('I', 'M')
('I', 'O')
('J', 'u')
('L', '</w>')
('L', 'A')
('L', 'l')
('M', 'O')
('M', 'a')
('M', 'i')
('M', 'u')
('N', 'I')
('O', '</w>')
('O', 'N')
('Q', 'u')
('R', 'A')
('R', 'E')
('R', 'R')
('R', 'e')
('S', '</w>')
('S', 'A')
('S', 'T')
('S', 'a')
('T', 'A')
('T', 'E')
('T', 'I')
('T', 'e')
('U', 'n')
('V', 'a')
('Y', ',')
('Y', '</w>')
('Y', 'o')
('a', ',')
('a', '.')
('a', '</w>')
('a', 'a')
('a', 'b')
('a', 'd')
('a', 'l')
('a', 'n')
('a', 'p')
('a', 'r')
('a', 's')
('a', 't')
('a', 'v')
('a', 'ñ')
('b', 'a')
('b', 'e')
('b', 'i')
('b', 'r')
('c', 'a')
('c', 'e')
('c', 'h')
('c', 'i')
('c', 'o')
('c', 'r')
('c', 't')
('c', 'u')
('d', ',')
('d', '</w>')
('d', 'a')
('d', '